### Imports

In [155]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import array as array
from scipy.stats import pearsonr
%matplotlib inline
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [156]:
### Function

In [157]:
def wAvrgAggr(df, cName):
    totalW = 0.0
    totVal = 0.0
    for value, weight in zip(df[cName], df['TotalPop']):
        totVal += float(value) * float(weight)
        totalW += weight
    return totVal/totalW

def sumAggregation(df, cName):
    return df[cName].sum()

def weighted_median(df, val):
    df_sorted = df.sort_values(val)
    cumsum = df_sorted['TotalPop'].cumsum()
    cutoff = df_sorted['TotalPop'].sum() / 2.
    return df_sorted[cumsum >= cutoff][val].iloc[0]

### Data Loading

In [158]:
demographics=pd.read_csv("data/income/demographic.csv")
c_results=pd.read_csv("data/output/countyData.csv")
s_results=pd.read_csv("data/output/stateData.csv")

### Data Cleaning - Demographics

In [159]:
#Porto Rico doesnt vote
demographics.drop(demographics[demographics.State=='Puerto Rico'].index, inplace=True)

#Drop Columns
demographics.drop(['CountyId','IncomeErr','IncomePerCapErr'], axis=1, inplace=True)

#General Data adjustment for mapping
demographics['County'] = demographics['County'].str.replace(' County', '')
demographics['County'] = demographics['County'].str.replace(' Parish', '')
demographics['County'] = demographics['County'].str.replace(' parish', '')

#Virginia
m = demographics['State'] == 'Virginia'
demographics.loc[m, 'County'] = demographics.loc[m, 'County'].replace({' city': ''}, regex=True)
demographics.loc[m, 'County'] = demographics.loc[m, 'County'].replace({' City': ''}, regex=True)

#Alaska
m = demographics['State'] == 'Alaska'
demographics.loc[m, 'County'] = 'Alaska'

#Sorting
demographics.sort_values(by=['State', 'County'], inplace=True, ascending=True)

### Data Cleaning - c_result

In [160]:
#Drop NA for Now until fix
c_results=c_results.dropna()

#TEMPORARY FIX /// Drop all Alaska Except one
c_results.drop(c_results[(c_results.state=='Alaska') & (c_results.county_id > 13)].index, inplace=True)


### Data Cleaning - s_result

In [161]:
#Drop NA for Now until fix
s_results=s_results.dropna()

### Data County Merge

In [163]:
County_merged=pd.merge(right=demographics, left=c_results, right_on=['State','County'], left_on=['state','county'])
County_merged.drop(['State', 'County'], axis=1, inplace=True)

#Alaska Fix
g = County_merged.groupby('state')
for state, group in g:
    if state == 'Alaska':
        # Note that TotalPop should be aggregated last, as it is used in the wAvrg
        sumAgg = ['Men','Women','TotalPop']
        wAvgAgg = ['IncomePerCap', 'Hispanic', 'White','Black','Native','Asian','Pacific', 'Poverty', 'ChildPoverty','Unemployment']
        wMedian = ['Income']
        for col in wMedian: 
            newVal = weighted_median(group, col)
            County_merged.loc[(County_merged['state'] == state), col] = newVal
        for col in wAvgAgg:  
            newVal = wAvrgAggr(group, col)
            County_merged.loc[(County_merged['state'] == state), col] = newVal
        for col in sumAgg:
            newVal = sumAggregation(group, col)
            County_merged.loc[(County_merged['state'] == state), col] = newVal
            
#Men/Women Ratio:
normalizedMen = []
normalizedWom = []
for (a, b) in zip(County_merged['Men'].astype("Int32"), County_merged['Women'].astype("Int32")):
    try:
        normalizedMen.append(a / (a + b))
        normalizedWom.append(b / (a + b))
    except:
        normalizedMen.append('')
        normalizedWom.append('')
County_merged['percentageMen'] = normalizedMen 
County_merged['percentageWomen'] = normalizedMen 

County_merged = County_merged[['id','county','county_id','state','state_id','DEM_votes','REP_votes','dem_votes_%', 'rep_votes_%', 'TotalPop','percentageMen', 'percentageWomen', 'Men','Women','Hispanic','White','Black','Native','Asian','Pacific','IncomePerCap', 'Income','Poverty','ChildPoverty','Unemployment']]
County_merged = County_merged.drop_duplicates()
m = County_merged['state'] == 'Alaska'
County_merged.loc[m, 'id'] = 2
County_merged.loc[m, 'county_id'] = 2
County_merged['state_id'] = County_merged['state_id'].astype("Int32")
County_merged['county_id'] = County_merged['county_id'].astype("Int32")
County_merged['id'] = County_merged['id'].astype("Int32")

County_merged.to_csv (r'data/output/Merged_County_V1.csv', index = False, header=True)

### Data State Merge

In [169]:
stateData = County_merged.copy()
g = stateData.groupby('state')
for state, group in g:
    # Note that TotalPop should be aggregated last, as it is used in the wAvrg
    sumAgg = ['Men','Women','TotalPop','DEM_votes','REP_votes']
    wAvgAgg = ['IncomePerCap', 'Hispanic', 'White','Black','Native','Asian','Pacific', 'Poverty', 'ChildPoverty','Unemployment']
    wMedian = ['Income']
    for col in wMedian: 
        newVal = weighted_median(group, col)
        stateData.loc[(stateData['state'] == state), col] = newVal
    for col in wAvgAgg:  
        newVal = wAvrgAggr(group, col)
        stateData.loc[(stateData['state'] == state), col] = newVal
    for col in sumAgg:
        newVal = sumAggregation(group, col)
        stateData.loc[(stateData['state'] == state), col] = newVal
    
#Men/Women Ratio:
normalizedMen = []
normalizedWom = []
for (a, b) in zip(stateData['Men'].astype("Int32"), stateData['Women'].astype("Int32")):
    try:
        normalizedMen.append(a / (a + b))
        normalizedWom.append(b / (a + b))
    except:
        normalizedMen.append('')
        normalizedWom.append('')
stateData['percentageMen'] = normalizedMen 
stateData['percentageWomen'] = normalizedWom

demNorm = []
repNorm = []
for (a, b) in zip(stateData['DEM_votes'].astype("Float32"), stateData['REP_votes'].astype("Float32")):
    try:
        demNorm.append((a / (a + b)))
    except:
        demNorm.append('')
    try:
        repNorm.append((b / (a + b)))
    except:
        repNorm.append('')
stateData['dem_votes_%'] = demNorm
stateData['rep_votes_%'] = repNorm

stateData = stateData[['state','state_id','DEM_votes','REP_votes','dem_votes_%', 'rep_votes_%', 'TotalPop','percentageMen', 'percentageWomen', 'Men','Women','Hispanic','White','Black','Native','Asian','Pacific','IncomePerCap', 'Income','Poverty','ChildPoverty','Unemployment']]
stateData = stateData.drop_duplicates()
stateData = stateData.sort_values(by=['state_id'])
stateData['state_id'] = stateData['state_id'].astype("Int32")
stateData = stateData.reset_index(drop=True)


In [170]:
# stateData= stateData.sort_values()
stateData.to_csv (r'data/output/Merged_State_V1.csv', index = False, header=True)
stateData

,state,state_id,DEM_votes,REP_votes,dem_votes_%,rep_votes_%,TotalPop,percentageMen,percentageWomen,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,IncomePerCap,Income,Poverty,ChildPoverty,Unemployment
0,Alabama,1,843473.0,1434159.0,0.370329,0.629671,4850771,0.484625,0.515375,2350806,2499965,4.090538,65.925899,26.394896,0.468075,1.271648,0.019180,25746.341350,46545,17.974093,25.944204,7.552098
1,Alaska,2,143037.0,179080.0,0.444053,0.555947,738565,0.523067,0.476933,386319,352246,6.812066,61.543834,3.025472,13.762591,6.061671,1.171835,35064.423431,76250,10.176225,13.532853,8.006877
2,Arizona,4,1672054.0,1661677.0,0.501556,0.498444,6809946,0.497075,0.502925,3385055,3424891,30.887945,55.588157,4.069675,3.888247,3.059985,0.174515,27963.445162,58580,16.952275,23.992275,7.209409
3,Arkansas,5,419258.0,758183.0,0.356076,0.643924,2977944,0.490826,0.509174,1461651,1516293,7.166490,73.007492,15.357785,0.572804,1.412328,0.227613,24426.218718,43504,18.178289,25.837504,6.361166
4,California,6,10760110.0,5750654.0,0.651703,0.348297,38982847,0.496797,0.503203,19366579,19616268,38.751217,37.909617,5.547530,0.356283,13.929404,0.346743,33127.865984,61015,15.118114,20.242040,7.768742
5,Colorado,8,1803873.0,1364160.0,0.569398,0.430602,5436519,0.502401,0.497599,2731315,2705204,21.287204,68.649807,3.873329,0.525543,2.967554,0.151255,34845.269500,64980,11.543312,14.712617,5.353002
6,Connecticut,9,106736.0,33713.0,0.759963,0.240037,3430368,0.487942,0.512058,1673819,1756549,15.798777,67.239052,9.992411,0.123753,4.361071,0.000000,41254.139681,69936,10.213096,13.757220,7.359870
7,Delaware,10,296268.0,200603.0,0.596267,0.403733,943732,0.484116,0.515884,456876,486856,8.994927,63.016373,21.359167,0.319068,3.816916,0.000000,32625.371345,68336,12.124656,17.477845,6.445333
8,District of Columbia,11,307265.0,18172.0,0.944161,0.055839,672391,0.474495,0.525505,319046,353345,10.700000,36.000000,46.900000,0.200000,3.700000,0.000000,50832.000000,77649,17.400000,25.500000,8.000000
9,Florida,12,5294767.0,5667474.0,0.483000,0.517000,20278447,0.488911,0.511089,9914361,10364086,24.747791,54.863623,15.423255,0.213137,2.633063,0.039642,28773.663009,51296,15.491542,22.249154,7.219904


In [ ]:
plt.scatter(stateData['IncomePerCap'], stateData['normalized_election_outcome']);

In [ ]:
County_merged